In [1]:
# ------------------------------------------
# IMPORT AND SETUP
# ------------------------------------------

import sys, os
import pandas as pd

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))

#Import the class RawData
from data_import import RawData
raw_data = RawData()

#Import the class RefinedData
from data_handling import RefinedData
ref_data = RefinedData()

#Import the class VisualTable
from utils import VisualTable
pretty_table = VisualTable()

"""
Functions within the class RawData, with arguments:

    get_met(
        self, 
        weather_station, 
        weather_elements, 
        weather_time, 
        weather_resolution
        )

    get_nilu(
        self, 
        threshold, 
        file_path
        )

Functions within the class RefinedData, with arguments:

    missing_data(
        self, 
        df, 
        strategy, 
        fill_value
        )
        
    show_zeroes(
        self, 
        df
        )
"""

'\nFunctions within the class RawData, with arguments:\n\n    get_met(\n        self, \n        weather_station, \n        weather_elements, \n        weather_time, \n        weather_resolution\n        )\n\n    get_nilu(\n        self, \n        threshold, \n        file_path\n        )\n\nFunctions within the class RefinedData, with arguments:\n\n    missing_data(\n        self, \n        df, \n        strategy, \n        fill_value\n        )\n        \n    show_zeroes(\n        self, \n        df\n        )\n'

In [2]:
# ------------------------------------------
# WEATHER DATA - IMPORT AND PROCESSING
# ------------------------------------------

# Variables that can be changed from run to run for the weather data
weather_station = 'SN68860' # The ID of the weather station to fetch data from, can be multiple stations separated by a comma - 'SN68860' is Voll weatherstation
weather_elements = 'mean(air_temperature P1D),sum(precipitation_amount P1D),mean(wind_speed P1D)' # The measurements to include in the dataset, here: temperautre, precipitation, and wind speed
weather_time = '2024-01-30/2024-03-19' # The stretch of time to fetch weather data for
weather_resolution = 'P1D' # The level of granularity in the weather data. P1D is once per day. P1H is once per hour.

# Run the module for importing the weather data, and save it to a dataframe
df_weather = raw_data.get_met(weather_station, weather_elements, weather_time, weather_resolution)

# Temporary weather data for when the API is down
'''df_weather = pd.DataFrame({
    'referenceTime': ['2024-01-30', '2024-01-31', '2024-02-01'],
    'Temperature': [10, 15, 11],
    'Precipitation': [0, 5, 2],
    'Wind_Speed': [3, 7, 2]
})'''

# Check if data was fetched successfully and check for missing values
if df_weather is None:
    print("Failed to fetch weather data.")
else:
    # Report missing values as a dataframe
    missing_df = ref_data.missing_data(df_weather, strategy='report')
    if missing_df is not None:
        print("Missing values found at these locations:")
        pretty_table.pretty_data(missing_df)

    # Dataframe without the rows containing missing values
    '''cleaned_df = ref_data.missing_data(df_weather, strategy='drop')
    if cleaned_df is not None:
        print("DataFrame after dropping missing values:")
        pretty_table.pretty_data(cleaned_df)'''

    # Fill missing values with chosen fill_value
    '''filled_df = ref_data.missing_data(df_weather, strategy='fill', fill_value=None)
    if filled_df is not None:
        print("DataFrame after filling missing values:")
        pretty_table.pretty_data(filled_df)'''

# Weather Data EDA
'''print("Weather Data Info:")
print(df_weather.info())
print("\nWeather Data Description:")
print(df_weather.describe())'''

# Displaying the weather data as a table
#pretty_table.pretty_data(df_weather)

Data collected from frost.met.no!
There are  244 lines of data in this dataframe.

No missing values found in the data set! 



'print("Weather Data Info:")\nprint(df_weather.info())\nprint("\nWeather Data Description:")\nprint(df_weather.describe())'

In [4]:
# ------------------------------------------
# AIR QUALITY DATA - IMPORT AND PROCESSING
# ------------------------------------------

# Variables that can be changed from run to run for the air quality data
threshold = 95 # The threshold below which we will not use the data for analysis (uptime % of day)
file_path = '../data/luftkvalitet_trondheim_dag.csv' #Choose the file to get air quality data from

# Run the module for importing the air quality data, and save it to a dataframe
df_quality = raw_data.get_nilu(threshold, file_path)

# Check if data was fetched successfully and check for missing values
if df_quality is None:
    print("Failed to fetch weather data.")
else:
    missing_data = ref_data.missing_data(df_quality)
    if missing_data is not None:
        print("\nThe following datapoints are missing:")
        pretty_table.pretty_data(missing_data)
    else:
        pass

poor_quality_data = ref_data.show_zeroes(df_quality)
if poor_quality_data is not None:
    print('\nThese datapoints contain no data or data of too poor quality:')
    pretty_table.pretty_data(poor_quality_data)
else:
    pass

# Air Quality Data EDA
'''print("\nAir Quality Data Info:")
print(df_quality.info())
print("\nAir Quality Data Description:")
print(df_quality.describe())'''

# Displaying the air quality data as a table
#pretty_table.pretty_data(df_quality)

Data collected from nilu.com!
There are  367 lines of data in this dataframe.


The following datapoints are missing:


,index,column
0,107,Elgeseter NO µg/m³ Day
1,107,Elgeseter NO2 µg/m³ Day
2,107,Elgeseter NOx µg/m³ Day
3,108,Elgeseter NO µg/m³ Day
4,108,Elgeseter NO2 µg/m³ Day
5,108,Elgeseter NOx µg/m³ Day
6,109,Elgeseter NO µg/m³ Day
7,109,Elgeseter NO2 µg/m³ Day
8,109,Elgeseter NOx µg/m³ Day
9,110,Elgeseter NO µg/m³ Day



These datapoints contain no data or data of too poor quality:


,index,column
0,18,Dekning
1,50,Dekning
2,57,Dekning
3,79,Dekning
4,107,Dekning
5,108,Dekning
6,109,Dekning
7,110,Dekning
8,111,Dekning
9,112,Dekning


'print("\nAir Quality Data Info:")\nprint(df_quality.info())\nprint("\nAir Quality Data Description:")\nprint(df_quality.describe())'

In [5]:
# ------------------------------------------
# SAVE DATA TO FILE
# ------------------------------------------



df_weather.to_csv('../data/refined_weather_data.csv', encoding='utf-8', index=False)

df_quality.to_csv('../data/refined_air_qualty_data.csv', encoding='utf-8', index=False)